**TO_DOs**:
1. Repasar que el `bm25` funcione bien (mirar cómo se calcula pa cada doc e ir printeando los parámetros que usa y verificar que todo esté bien). Pd: el ranking ya funciona pero eso no quiere decir que no la haya liado en un parámetro o dos.
2. Create "our own score" --> qué putas
3. Cuando hagamos el print_top_20 pa comparar molaría tunear un poco el código pa que fuese aun más "visualmente comparable". Ahora mismo está simplemente en dos celdas separadas y es poco comparable.
1000. Report-> explicar que hemos reutilizado código para el bm25 pero que necesitabamos cosas nuevas y que por eso hemos creado un diccionario docu_length y el lavg etc. Explicar que hemos usado k1=1.6 y b=0.75 que lo ponia en la teoria. Y que vemos claramente que tiene más en cuenta docu_length que el tf-idf y que para nosotros es mejor.

In [2]:
# ----------------------------------------------------------------------------------- #
# SON LOS IMPORTS DEL LAB ANTERIOR, LOS DESCOMENTAMOS A MEDIDA QUE LOS VAYAMOS USANDO #
# ----------------------------------------------------------------------------------- #

# from collections import defaultdict
from array import array
import nltk
#nltk.download('stopwords')
# # nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
import math
import numpy as np
import collections
from numpy import linalg as la
# import json
# import random
# import re
import pandas as pd
import time

### Loading the processed tweets

In [3]:
output_folder = '../output/'
data_folder = '../data/'
data = pd.read_csv(output_folder + "lab1_tweets_df.csv", sep='|')
data.head(2)

,Document,Id,Text,Username,Date,Hashtags,Mentions,Likes,Retweets,Url,Clean_text
0,doc_1,1575918182698979328,So this will keep spinning over us until 7 pm…...,suzjdean,Fri Sep 30 18:39:08 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/suzjdean/status/1575918182...,"['keep', 'spin', 'us', 'away', 'alreadi', 'hur..."
1,doc_2,1575918151862304768,Our hearts go out to all those affected by #Hu...,lytx,Fri Sep 30 18:39:01 +0000 2022,['HurricaneIan'],NaN,0,0,https://twitter.com/lytx/status/15759181518623...,"['heart', 'go', 'affect', 'hurricaneian', 'wis..."


# Indexing and Ranking definition

## Inverted index

In [4]:
def build_terms(line):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
    line=  line.lower() ## Transform in lowercase
    line=  line.split() ## Tokenize the text to get a list of terms
    line= [x for x in line if x not in stop_words]  ##eliminate the stopwords (HINT: use List Comprehension)
    line= [stemmer.stem(x) for x in line] ## perform stemming (HINT: use List Comprehension)
    ## END CODE
    return line

In [5]:
def create_index_tfidf(dataframe):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    dataframe -- DataFrame containing tweet information
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    num_documents = dataframe.shape[0]
    index = defaultdict(list)
    tf = defaultdict(list)  # term frequencies of terms in documents (documents in the same order as in the main index)
    df = defaultdict(int)   # document frequencies of terms in the corpus
    idf = defaultdict(float)
    for row in dataframe.iterrows():
        doc_id = row[1]['Document']
        terms = row[1]['Clean_text']  
        terms = eval(terms)
        
        ## ===============================================================        
        ## create the index for the **current page** and store it in current_page_index
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}

        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms):  ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position) 
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[doc_id, [position]]
                
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm = 0
        for term, posting in current_page_index.items():
            # posting will contain the list of positions for current term in current document. 
            # posting ==> [current_doc, [list of positions]] 
            # you can use it to infer the frequency of current term.
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        # calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4)) ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term] = df[term]+1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)

    return index, tf, df, idf


In [6]:
start_time = time.time()
new_index, tf, df, idf = create_index_tfidf(data)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the index: 429.06 seconds


## Test queries

### Words analysis in the collection:

In [7]:
## ============================= ##
## show top DF in the collection ##
## ============================= ##

word = []
word_df = []
for item in df.items():
    word.append(item[0])
    word_df.append(item[1])

document_frequency_df = pd.DataFrame(list(zip(word,word_df)), columns=['Word', 'DF'])
document_frequency_df.sort_values(by='DF', ascending=False).head(20)

,Word,DF
5,hurricaneian,3988
72,florida,881
47,hurrican,793
70,ian,781
211,help,386
122,amp,362
25,storm,352
87,carolina,297
303,flood,289
293,power,275


Based docuemnt frequency of the terms above, we define the following queries for our test cases: 

In [8]:
queries = [
    "Floodings in South Carolina", 
    "HurracaineIan disaster", 
    "Damage of HuracaineIan", 
    "Florida floodings", 
    "Storm and wind in Florida"
]

## Ranking

New **function** `BestMatch25`:

Formula: $RSV_d = \sum_{t \in q} log [\frac{N}{dft}] * \frac{(k+1)*tf_{td}}{k1*((1-b) + b*(\frac{L_d}{L_{ave}}) ) + tfd} $

Because we will need `Ld` and `Lavg`, we will build a dictionnary with documents and their corresponding lenghts, and with it we will calculate Lave.

In [9]:
docu_length = dict()
for index, row in data.iterrows():
    docu_length[row['Document']] = len(eval(row['Clean_text']))

In [10]:
lavg = 0
for length in docu_length.values():
    lavg += length
lavg = round(lavg / len(docu_length))
print(lavg)

15


In [11]:
def rank_documents_bm25 (terms, docs, index, idf, tf, k1, b, N, docu_length, lavg):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    
    Returns:
    list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary


    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if len(index[term])==0:
            continue
        
        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):         
            if doc in docs:
                dft = len(index[term]) #in how many documents does it appear
                ld = docu_length[doc] #document length
                doc_vectors[doc][termIndex] = np.log(N/dft) * (((k1+1)*tf[term][doc_index]) / (k1*((1-b)+b*( ld / lavg ) ) + tf[term][doc_index]))
    
    # Calculate the score of each doc (the sum of the results)    
    doc_scores=[[np.sum(curDocVec), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(doc_scores) == 0:
        print("No results found for query")
    #print ('\n'.join(result_docs), '\n')
    return doc_scores

In [12]:
#Let's keep that as a reference
def rank_documents(terms, docs, index, idf, tf):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    
    Returns:
    list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex]=query_terms_count[term]/query_norm *idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):

            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]
    
    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]
    if len(doc_scores) == 0:
        print("No results found for query")
    #print ('\n'.join(result_docs), '\n')
    return doc_scores

In [186]:
def search_tf_idf(query, index, ranking):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    if (ranking == "bm25"):
        ranked_docs = rank_documents_bm25(query, docs, index, idf, tf, k1 = 1.6, b = 0.75, N = data.shape[0], docu_length = docu_length, lavg = lavg) #TO-DO new formula!! (new function, new attributes)
    elif (ranking == "tf-idf_cosine-similarity"):
        ranked_docs = rank_documents(query, docs, index, idf, tf)
    else:
        print("We don't have this way of ranking... Maybe try with: \n 1) bm25 \n 2) classical-tf-idf")
    return ranked_docs

# 1. **Ranking results**

## a) Ranking for `TF-IDF + cosine similarity`

In [188]:
print(f"{' RANKING WITH TF-IDF + Cosine similarity ':=^80}")
for query in queries:
    ranked_docs = search_tf_idf(query, new_index, ranking ="tf-idf_cosine-similarity")
    top = 5
    print(f"{'='}" * 80)
    print(f"\nTop {top} results out of {len(ranked_docs)} for the searched query '{query}':\n")
    for rank, doc in enumerate(ranked_docs[:top], start=1):
        print(f"{rank:=^80}")
        print(f"{doc[1]:^80} \n\t- Score: {doc[0]}\n\t- Text:  {data[data['Document']==doc[1]]['Clean_text'].iloc[0]}")

=================== RANKING WITH TF-IDF + Cosine similarity ====================

Top 5 results out of 605 for the searched query 'Floodings in South Carolina':

=======================================1========================================
                                    doc_254                                      
	- Score: 4.74475387696339
	- Text:  ['south', 'carolina', 'hurricaneian']
=======================================2========================================
                                    doc_1289                                     
	- Score: 4.315205653041319
	- Text:  ['flood', 'garden', 'citi', 'south', 'carolina', 'gardenc', 'southcarolina', 'hurricaneian']
=======================================3========================================
                                    doc_2874                                     
	- Score: 4.10872348195652
	- Text:  ['charleston', 'south', 'carolina', 'hurricaneian']
=======================================4==============

## b) Ranking for `BM25`

In [189]:
print(f"{' RANKING WITH BM25 ':=^80}")
for query in queries:
    ranked_docs = search_tf_idf(query, new_index, ranking ="bm25")
    top = 5
    print(f"{'='}" * 80)
    print(f"\nTop {top} results out of {len(ranked_docs)} for the searched query '{query}':\n")
    for rank, doc in enumerate(ranked_docs[:top], start=1):
        print(f"{rank:=^80}")
        print(f"{doc[1]:^80} \n\t- Score: {doc[0]}\n\t- Text:  {data[data['Document']==doc[1]]['Clean_text'].iloc[0]}")

============================== RANKING WITH BM25 ===============================

Top 5 results out of 605 for the searched query 'Floodings in South Carolina':

=======================================1========================================
                                    doc_254                                      
	- Score: 6.577248263193381
	- Text:  ['south', 'carolina', 'hurricaneian']
=======================================2========================================
                                    doc_2874                                     
	- Score: 5.683435379919324
	- Text:  ['charleston', 'south', 'carolina', 'hurricaneian']
=======================================3========================================
                                    doc_2834                                     
	- Score: 5.683435379919324
	- Text:  ['charleston', 'south', 'carolina', 'hurricaneian']
=======================================4========================================
            

## b) Ranking for `(own score)`

# 2. **Word2Vec with cosine similarity**

In this part we will rank our documents using cosine similarity. The thing is that instead of TF-IDF vector representation of the queries we will use word2vec.

First we train the `model` using our dataset. 

In [178]:
from ast import literal_eval
from gensim.models import Word2Vec
words = []
for _, tweet in data.iterrows():
    words.append(literal_eval(tweet['Clean_text']))
model = Word2Vec(words, min_count = 1, vector_size = 100, window = 5)

Build a `tweet2vec` representation for each document.

In [179]:
def tweet2vec_builder(model):
    """
    Build a tweet2vec dictionary containing the average of the word2vec vectors for each document.

    Argument:
    model -- pretrained model 

    Returns:
    dictionary with the vector representation of each doc
    """
    tweet2vec = {}
    for _, tweet in data.iterrows():
        doc_vec = [model.wv[term] for term in literal_eval(tweet['Clean_text'])]
        tweet2vec[tweet['Document']] = np.mean(doc_vec, axis=0)
    return tweet2vec

In [180]:
tweet2vec = tweet2vec_builder(model)

Computing the cosine similarity between pairs of vector representations for **queries** and **tweets**.

In [181]:
def word2vec_cosine_similarity(query, docs, tweet2vec, model):
    """
    Rank documents based on the cosine similarity with the query.
    
    Argument:
    query -- list of terms in the query
    docs -- documents containing the query terms
    tweet2vec -- dictionary containing the tweet2vec representation of each document
    model -- pretrained word2vec model
    
    Returns:
    list of ranked documents
    """

    # Query word2vec matrix. Contains len(terms) word2vec vectors.
    query_vec = np.array([model.wv[term] for term in query if term in model.wv.key_to_index])
    # Averaging all the word2vec of the query to optain query2vec
    query_vec = np.mean(query_vec, axis=0)

    # Cosine similarity between query2vec and tweet2vec (which have the same dimension)
    doc_scores=[[np.dot(doc_vec, query_vec), doc] for doc, doc_vec in tweet2vec.items() if doc in docs]
    doc_scores.sort(reverse=True)
    if len(doc_scores) == 0:
        print("No results found for query")
    return doc_scores

In [182]:
def search_word2vec(query, tweet2vec, model, index):
    """
    Preprocess the query, find the docs containing the query terms and executing the ranking.
    
    Argument:
    query -- string
    tweet2vec -- dictionary containing the tweet2vec representation of each document
    model -- pretrained word2vec model
    index -- inverted index data structure
    
    Returns:
    list of ranked documents
    """
    
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = word2vec_cosine_similarity(query, docs, tweet2vec, model)
    return ranked_docs
    


## **Ranking** for `Word2Vec + Cosine similarity`

In [183]:
print(f"{' RANKING WITH Word2Vec + Cosine similarity ':=^80}")
for query in queries:
    ranked_docs = search_word2vec(query, tweet2vec, model, new_index)
    top = 20
    print(f"{'='}" * 80)
    print(f"\nTop {top} results out of {len(ranked_docs)} for the searched query '{query}':\n")
    for rank, doc in enumerate(ranked_docs[:top], start=1):
        print(f"{rank:=^80}")
        print(f"{doc[1]:^80} \n\t- Score: {doc[0]}\n\t- Text:  {data[data['Document']==doc[1]]['Clean_text'].iloc[0]}")

================== RANKING WITH Word2Vec + Cosine similarity ===================

Top 20 results out of 605 for the searched query 'Floodings in South Carolina':

=======================================1========================================
                                    doc_2901                                     
	- Score: 28.111572265625
	- Text:  ['hurricaneian', 'florida', 'rain', 'flood', 'report']
=======================================2========================================
                                    doc_254                                      
	- Score: 26.291181564331055
	- Text:  ['south', 'carolina', 'hurricaneian']
=======================================3========================================
                                    doc_174                                      
	- Score: 26.28900718688965
	- Text:  ['south', 'carolina', 'hurricaneian', 'go']
=======================================4========================================
                 

# 3. **Better representations?** (discussion)